In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv('sick.csv', index_col=False)
data.drop('Unnamed: 0',axis=1, inplace=True)
data

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Effusion,Emphysema,Fibrosis,Infiltration,Mass,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,pathology_vec
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143000,...,0,0,0,0,0,0,0,0,0,[0 1 0 0 0 0 0 0 0 0 0 0 0]
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143000,...,0,1,0,0,0,0,0,0,0,[0 1 0 0 0 1 0 0 0 0 0 0 0]
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168000,...,1,0,0,0,0,0,0,0,0,[0 1 0 0 1 0 0 0 0 0 0 0 0]
3,00000004_000.png,Mass|Nodule,0,4,82,M,AP,2500,2048,0.168000,...,0,0,0,0,1,1,0,0,0,[0 0 0 0 0 0 0 0 1 1 0 0 0]
4,00000005_006.png,Infiltration,6,5,70,F,PA,2992,2991,0.143000,...,0,0,0,1,0,0,0,0,0,[0 0 0 0 0 0 0 1 0 0 0 0 0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51519,00030786_007.png,Consolidation|Pleural_Thickening,7,30786,61,F,AP,3056,2544,0.139000,...,0,0,0,0,0,0,1,0,0,[0 0 1 0 0 0 0 0 0 0 1 0 0]
51520,00030789_000.png,Infiltration,0,30789,52,F,PA,2021,2021,0.194311,...,0,0,0,1,0,0,0,0,0,[0 0 0 0 0 0 0 1 0 0 0 0 0]
51521,00030793_000.png,Mass|Nodule,0,30793,58,F,PA,2021,2021,0.194311,...,0,0,0,0,1,1,0,0,0,[0 0 0 0 0 0 0 0 1 1 0 0 0]
51522,00030795_000.png,Pleural_Thickening,0,30795,53,F,PA,2021,2021,0.194311,...,0,0,0,0,0,0,1,0,0,[0 0 0 0 0 0 0 0 0 0 1 0 0]


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, 
                                   test_size = 0.25)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_gen = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              height_shift_range= 0.05, 
                              width_shift_range=0.1, 
                              rotation_range=5, 
                              shear_range = 0.1,
                              fill_mode = 'constant',
                              zoom_range=0.15)

In [ ]:
image_size = (128, 128)

train_gen = img_gen.flow_from_dataframe(dataframe=train_df, x_col = 'path',
y_col = 'clean_labels', classmode = 'multi_output',
classes = clean_labels, target_size = image_size, color_mode = 'grayscale',
batch_size = 32)

test_gen = img_gen.flow_from_dataframe(dataframe=test_df, x_col = 'path',
y_col = 'clean_labels', classmode = 'multi_output',
classes = clean_labels, target_size = image_size, color_mode = 'grayscale',
batch_size = 32)

test_X, test_Y = next(img_gen.flow_from_dataframe(dataframe=test_df,
x_col = 'path', y_col = 'clean_labels',
classmode = 'multi_output', classes = clean_labels,
target_size = image_size,
color_mode = 'grayscale', batchsize = 1024))
                            

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('xray_class')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=3)
callbacks_list = [checkpoint, early]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(128, (2, 2), input_shape=(128,128,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense((len(clean_labels))))
model.add(Activation('sigmoid'))

model.load_weights('xray_class_unflip_model_weights.best.hdf5')

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['categorical_accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_gen, 
                              steps_per_epoch=train_df.shape[0]//32,
                              validation_data = (test_X, test_Y), 
                              epochs = 10, 
                              callbacks = callbacks_list)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
